# **Import Dataset**

In [ ]:
!pip3 install pickle5
import pickle5 as pc

import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import pickle
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline
from datetime import datetime
import tensorflow as tf

# set random seed for random weights
np.random.seed(1)
tf.random.set_seed(1)
fd=''
fk='../../data'
with open(fd + fk + '/data_train_v1', "rb") as fh:
  data_train = pc.load(fh).reset_index().iloc[:,1:]
with open(fd + fk +'/data_test_v1', "rb") as fh:
  data_test = pc.load(fh).reset_index().iloc[:,1:]

#data_train[data_train.ID == 1]

fig, ax = plt.subplots(figsize=(7,5))

ax.plot(data_train[data_train.ID == 1]['cycle'],data_train[data_train.ID == 1]['gauge1'],label= 'gauge1')
ax.plot(data_train[data_train.ID == 1]['cycle'],data_train[data_train.ID == 1][ 'gauge2'],label= 'gauge2')
ax.plot(data_train[data_train.ID == 1]['cycle'],data_train[data_train.ID == 1]['gauge3'], label= 'gauge3')
ax.grid()
ax.legend()

ax.set(xlabel='Cycle', ylabel='Strain')

plt.savefig('training_data_ID_1.pdf', bbox_inches='tight')

plt.show()

plt.figure(figsize=(20, 12))
plt.plot(data_test[data_test.ID == 1]['gauge1'],label= 'gauge1')
plt.plot(data_test[data_test.ID == 1][ 'gauge2'],label= 'gauge2')
plt.plot(data_test[data_test.ID == 1]['gauge3'], label= 'gauge3')
plt.grid()
plt.legend()
plt.show()

# **Categorize Data**

In [ ]:
print(data_train.RUL.describe())
print(data_test.RUL.describe())

l=0.5
nb_bins = 20
lower_bin_bound = 0
upper_bin_bound = 80000

bins = np.linspace(lower_bin_bound, upper_bin_bound**(1-l), nb_bins)**(1/l)
labels=[i for i in range(bins.shape[0]-1)]
plt.plot(bins,np.full(bins.shape[0],0),'o')
print(bins)
print(labels)

data_train['RUL_bins'] = pd.cut(data_train['RUL'], bins=bins, labels=labels)
data_test['RUL_bins'] = pd.cut(data_test['RUL'], bins=bins, labels=labels)

display(data_train)
display(data_test)

# **Build Sequences**

In [ ]:
#utils 
nb_gauges = 3
data_tr = data_train[data_train.ID <= 100]   #500, 1000
data_val = data_test[data_test.ID <= 100]

#prepare forecasting data
def gen_X_sequence(id_df, seq_length, seq_cols,timesteps_pred,  type_data = None):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    
    
    ind_start = 0
    
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0+ind_start, num_elements-seq_length+1-timesteps_pred), range(seq_length+ind_start, num_elements+1-timesteps_pred)):
        yield data_array[start:stop, :]
 

def gen_Y_sequence(id_df, seq_length, seq_cols,timesteps_pred, type_data = None):
    """ Only sequences that meet the window-length are considered, no padding is used. This means for testing
    we need to drop those which are below the window-length. An alternative would be to pad sequences so that
    we can use shorter ones """
    
    
    ind_start = 0
    
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(range(0+ind_start, num_elements-seq_length+1-timesteps_pred), range(seq_length+ind_start, num_elements+1-timesteps_pred)):
        yield data_array[stop-1, :]#data_array[start+1:stop+1, :]
   
#prepare data
seq_cols =  ['gauge'+str(i) for i in range(1,4)]#['label'+str(i) for i in range(1,4)]
seq_cols1 =  ['RUL_bins']
sequence_length = 30
timesteps_pred = 1



#training set
seq_gen = (list(gen_X_sequence(data_tr[data_tr['ID']==id], sequence_length, seq_cols, timesteps_pred, type_data= 'train')) 
                   for id in data_tr['ID'].unique())
# generate sequences and convert to numpy array
dbX = np.concatenate(list(seq_gen))



seq_gen = (list(gen_Y_sequence(data_tr[data_tr['ID']==id], sequence_length, seq_cols1, timesteps_pred, type_data= 'train')) 
                   for id in data_tr['ID'].unique())
# generate sequences and convert to numpy array
dbY = np.concatenate(list(seq_gen))#.astype(np.long).reshape(-1,)


print(dbX.shape)
print(dbY.shape)

# validation set
seq_gen = (list(gen_X_sequence(data_val[data_val['ID']==id], sequence_length, seq_cols, timesteps_pred, type_data= 'train')) 
                   for id in data_val['ID'].unique())

# generate sequences and convert to numpy array
dbX_val = np.concatenate(list(seq_gen))#.astype(np.long)

seq_gen = (list(gen_Y_sequence(data_val[data_val['ID']==id], sequence_length, seq_cols1, timesteps_pred, type_data= 'train')) 
                   for id in data_val['ID'].unique())
# generate sequences and convert to numpy array
dbY_val = np.concatenate(list(seq_gen))#.astype(np.long).reshape(-1,)

#test set
seq_gen = (list(gen_X_sequence(data_test[data_test['ID']==id], sequence_length, seq_cols, timesteps_pred, type_data= 'train')) 
                    for id in data_test['ID'].unique())
# generate sequences and convert to numpy array
dbX_test = np.concatenate(list(seq_gen))

seq_gen = (list(gen_Y_sequence(data_test[data_test['ID']==id], sequence_length, seq_cols1, timesteps_pred, type_data= 'train')) 
                    for id in data_test['ID'].unique())
# generate sequences and convert to numpy array
dbY_test = np.concatenate(list(seq_gen))



print(dbX_val.shape)
print(dbY_val.shape)

# **Normalize Data**

In [ ]:

print(dbX.shape)
dbX[:,:,0].max()
for k in range(3) :
  a = dbX[:,:,k].max()
  b = dbX[:,:,k].min()
  dbX[:,:,k] = (dbX[:,:,k] - b)/(a-b)
  dbX_val[:,:,k] = (dbX_val[:,:,k] - b)/(a-b)
  dbX_test[:,:,k] = (dbX_val[:,:,k] - b)/(a-b)


dbX
dbX_val
# train_dataset = tf.data.Dataset.from_tensor_slices((dbX, dbY)).batch(batch_size)
# val_dataset = tf.data.Dataset.from_tensor_slices((dbX_val, dbY_val)).batch(batch_size)
# test_dataset = tf.data.Dataset.from_tensor_slices((dbX_test, dbY_test)).batch(batch_size)


# **Build the model**

In [ ]:
dbY.shape

In [ ]:
!pip install keras-tuner

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import kerastuner
from kerastuner.tuners import *
from kerastuner import HyperModel
from keras.wrappers.scikit_learn import KerasClassifier



class MyHyperModel(HyperModel):

    def __init__(self, input_shape, output_shape):
        self.input_shape = input_shape
        self.output_shape = output_shape

    def build(self, hp):
        # build model
        input_layer = Input(shape=(dbX.shape[1],dbX.shape[2]))
        x = LayerNormalization(axis=1)(input_layer)

        for i in range(hp.Int('num_layers', min_value=0, max_value=2, step=1)):
            x = GRU(units=hp.Choice('units', values=[32, 64, 128, 256]), 
                     dropout=hp.Choice('Dropout',values=[0.0, 0.1]),
                     recurrent_dropout=hp.Choice('Recurrent_Dropout',values=[0.0, 0.1]), return_sequences=True)(x)
        x = GRU(units=hp.Choice('units', values=[32, 64, 128, 256]), 
                 dropout=hp.Choice('Dropout',values=[0.0, 0.1]), 
                 recurrent_dropout=hp.Choice('Recurrent_Dropout',values=[0.0, 0.1]))(x)
        x = Activation('relu')(x)

        x = Dense(20, activation='softmax')(x)
        output_layer = x

        model = Model(input_layer, output_layer)

        # compile model
        model.compile(
            optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
            loss='sparse_categorical_crossentropy',
            metrics=['SparseCategoricalAccuracy'])

        return model


# **Compile and train**

In [ ]:
dbY.shape

In [ ]:
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from tensorflow.keras.initializers import *
from kerastuner.tuners import *
from kerastuner import HyperModel
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV



batch_size = 4096
sequence_length = 15


# # Create a MirroredStrategy.
# strategy = tf.distribute.MirroredStrategy()
# print("Number of devices: {}".format(strategy.num_replicas_in_sync))

# # Open a strategy scope.
# with strategy.scope():
#     # Everything that creates variables should be under the strategy scope.
#     # In general this is only model construction & `compile()`.
hypermodel = MyHyperModel(input_shape = (dbX.shape[0], dbX.shape[1]), output_shape = dbY.shape[1])
tuner = RandomSearch(
    hypermodel,
    overwrite=True,
    objective='val_sparse_categorical_accuracy',
    max_trials=20)

tuner.search_space_summary()


val_dataset=tf.data.Dataset.from_tensor_slices((dbX_val, dbY_val)).batch(batch_size)
train_dataset=tf.data.Dataset.from_tensor_slices((dbX, dbY)).batch(batch_size)

tuner.search(train_dataset,
             epochs=200,
             verbose=2,
             validation_data=val_dataset,
             #callbacks=[tf.keras.callbacks.TensorBoard(tuner.directory + '/' + tuner.project_name)],
             )

models = tuner.get_best_models(num_models=1)

tuner.results_summary()

best_model = models[0]

model_path = tuner.directory + '/' + tuner.project_name + '/' + 'best_model'

# get model as json string and save to file
model_as_json = best_model.to_json()
with open(model_path + '.json', "w") as json_file:
    json_file.write(model_as_json)
# save model weights
best_model.save_weights(model_path + '_weights.h5')



  

# **Evaluate**

In [ ]:
model.evaluate(dbX_val, dbY_val, batch_size=32)
# summarize history for accuracy
acc = List_hist[0] + List_hist[4] + List_hist[8]        # concatenate history for the 3 learning rate to make it a single learning
val_acc = List_hist[1] + List_hist[5] + List_hist[9]
loss = List_hist[2] + List_hist[6] + List_hist[10]
val_loss = List_hist[3] + List_hist[7] + List_hist[11]
plt.plot(acc)
plt.plot(val_acc)
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(loss)
plt.plot(val_loss)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()